1. Keeping only residential units?
2. 'within' while merging or 'intersects'?
3. counting only residential units or all?

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

parcels = gpd.read_file("../00_source_data/bladen-parcels")
tracts_demo= gpd.read_file("../00_source_data/2020_counts_NC_tract.csv")
tracts_boundaries= gpd.read_file("../00_source_data/tract_polygons/tl_2020_37_tract.shp")
tracts_boundaries=tracts_boundaries[tracts_boundaries.COUNTYFP=="017"]
tracts_boundaries.columns
##data dict: https://www.arcgis.com/sharing/rest/content/items/4b5f3f5c54b14339b81d8ff1a841dc2e/info/metadata/metadata.xml?format=default&output=html

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [2]:
col_names = {
    "ALTPARNO": "Alternate Parcel Number",
    "CNTYFIPS": "County FIPS Code",
    "CNTYNAME": "County Name",
    "GISACRES": "GIS Acres",
    "GNISID": "GNIS Identifier",
    "IMPROVVAL": "Improved Value",
    "LANDVAL": "Land Value",
    "LEGDECFULL": "Full Legal Description",
    "MAILADD": "Full Mailing Address",
    "MAPREF": "Map Reference",
    "MULTISTRUC": "Multiple Structures",
    "NPARNO": "National Parcel Number",
    "OWNNAME": "Owner Name",
    "PARNO": "Parcel ID",
    "OWNTYPE": "Owner Type",
    "PARUSECODE": "Parcel Use Code",
    "PARUSECD2": "Structure Use Code",
    "PARUSEDESC": "Parcel Use Description",
    "PARUSEDSC2": "Secondary Parcel Use Description",
    "PARVAL": "Total Parcel Value",
    "PRESENTVAL": "Present Use Value",
    "RECAREANO": "Land Area",
    "SADDSTR": "Site Address Street Name",
    "SALEDATETX": "Last Sale Date",
    "SITEADD": "Full Site Address",
    "SOURCEDATX": "Source Document Date",
    "SOURCEREF": "Source Document",
    "STRUCT": "Structure or Improvment",
    "TRANSFDATE": "Transformation Date",
}
drop_cols = [
    "MADDPREF",
    "MADDRNO",
    "MADDSTNAME",
    "MADDSTR",
    "MADDSTSUF",
    "MADDSTTYP",
    "MCITY",
    "MSTATE",
    "MZIP",
    "MUNIT",
    "OWNFRST",
    "OWNLAST",
    "OWNNAME2",
    "PARVALTYPE",
    "REVDATETX",
    "REVISEDATE",
    "REVISEYEAR",
    "SADDNO",
    "SADDPREF",
    "SADDSTNAME",
    "SADDSTSUF",
    "SADDSTTYP",
    "SALEDATE",
    "SCITY",
    "SOURCEAGNT",
    "SOURCEDATE",
    "SSTATE",
    "STCNTYFIPS",
    "STFIPS",
    "STNAME",
    "STRUCTNO",
    "STRUCTYEAR",
    "SUBDIVISIO",
    "SUBOWNTYPE",
    "SUBSURFOWN",
    "SUNIT",
    "SZIP",
]
parcels = parcels.rename(col_names, axis=1)
parcels = parcels.drop(drop_cols, axis=1)
#keeping residential parcels
# parcels = parcels[parcels["Parcel Use Description"].isin(["RESIDENTIAL IMPROVED"])] 

In [3]:
#checking geometry details
tracts_boundaries.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [4]:
#checking geometry details
parcels.crs
##crs for the two datasets don't match up

<Projected CRS: PROJCS["NAD83 / North Carolina (ftUS)",GEOGCS["NAD ...>
Name: NAD83 / North Carolina (ftUS)
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [5]:
parcels = parcels.to_crs(epsg=4269) #making datasets compatible for merging
## merging parcel points with polygons
parcel_shapes= gpd.sjoin(parcels, tracts_boundaries, op='within') 
print(f"There are {parcel_shapes.shape[0]} parcels and {parcel_shapes.GEOID.nunique()} tracts in Bladen County")

/Users/rachelrichards/opt/miniconda3/envs/geo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


There are 33146 parcels and 11 tracts in Bladen County


In [6]:
#creating merged dataset for estimates
est_df=parcel_shapes.merge(tracts_demo, left_on= "GEOID",right_on="full_fips", how="left")
est_df.head()

In [ ]:
est_df["Secondary Parcel Use Description"].value_counts()

SINGLE FAMILY DWELLING       8956
MOBILE HOME (REAL)           1483
CHURCH                        203
RETAIL STORE                  190
OFFICE                        116
WAREHOUSE                      73
APARTMENT TOWNHOUSE            68
SERVICE GARAGE                 47
CONDOMINIUM                    45
COMMUNITY RECREATION BUIL      33
APARTMENT FLAT                 28
INDUSTRIAL BUILDING - LIG      27
DUPLEX/TRIPLEX                 24
CLASSROOM BUILDING             22
MOTEL                          22
CONVENIENCE STORE              21
RESTAURANT                     21
MEDICAL OFFICE                 21
FIRE/RESCUE STATION            21
SHOP                           20
BEAUTY/BARBER SHOP             15
BANK                           12
LODGE                          10
GOVERNMENT BUILDING             9
SERVICE STATION                 8
COMMUNITY SERVICE BUILDIN       8
DAY CARE CENTER (NURSERY)       8
FUNERAL HOME                    8
FAST FOOD RESTAURANT            7
LAUNDROMAT    

In [ ]:
##Generating estimates for a single tract. refer to dictionary to see how units were counted
est_dict={'SINGLE FAMILY DWELLING':1, 'MOBILE HOME (REAL)':1, 'APARTMENT TOWNHOUSE':1, 'APARTMENT FLAT ':1,'DUPLEX/TRIPLEX':2,'CONDOMINIUM':1} ##dictionary to count housing units
est_df['housing_est']= [est_dict[x] if x in est_dict.keys() else 0 for x in est_df["Secondary Parcel Use Description"]] #replacing values with dictionary mappings
#grouping by tract
perc_diff_df= pd.DataFrame(est_df[["full_fips","total_housing"]].groupby("full_fips", sort=False)["total_housing"].unique()).reset_index()
perc_diff_df2=pd.DataFrame(est_df[["full_fips","housing_est"]].groupby("full_fips", sort=False)["housing_est"].sum()).reset_index()
#calculating percentage difference
perc_diff_df['total_housing']=[float(x[0]) for x in perc_diff_df['total_housing']]
perc_diff_df=perc_diff_df.merge(perc_diff_df2)
perc_diff_df['diff']= ((perc_diff_df["housing_est"] - perc_diff_df["total_housing"])/perc_diff_df["total_housing"])*100
perc_diff_df

,full_fips,total_housing,housing_est,diff
0,37017950302,1505.0,965,-35.880399
1,37017950200,1279.0,817,-36.121970
2,37017950101,1074.0,767,-28.584730
3,37017950102,2152.0,1421,-33.968401
4,37017950301,1118.0,674,-39.713775
5,37017950401,1732.0,1338,-22.748268
6,37017950402,909.0,653,-28.162816
7,37017950501,1092.0,812,-25.641026
8,37017950601,1410.0,1088,-22.836879
9,37017950502,1686.0,1205,-28.529063
